In [1]:
import re
import os
import sys
import datetime
import csv
import pandas as pd
import numpy as np
import datetime as dt
from dateutil.parser import parse

In [2]:
os.curdir
# os.chdir("C:/Dropbox/Berkeley MIDS/MIDS+/Tusome/TusomeOps/Mpesa reformatter")
# os.curdir
os.chdir("C:/Dropbox/Berkeley MIDS/MIDS+/Tusome/TusomeOps/Mpesa reformatter")

In [3]:
bulk_plan = pd.read_csv("M-pesa bulk payment.csv", sep = ",", skiprows = 3, nrows = 1, usecols = ["Bulk Plan Name", "Bulk Plan Description"])
bulk_plan_dets = bulk_plan.iloc[0,0] + "-" + bulk_plan.iloc[0,1]

In [4]:
columns = ["Record No", "Validation Result", "Transaction Timestamp", "Finished Timestamp", "TransactionID",
           "Transaction Details", " Amount ", "Fee Charge", "Extra Fee Charge", "Status", "Error Code", "Error Message"]
df = pd.DataFrame(pd.read_csv("M-pesa bulk payment.csv", sep = ",", header=12,
                              usecols = columns, index_col = "Record No",
                             parse_dates = ["Transaction Timestamp"]))

In [5]:
cols_corrected = {"Record No": "record_num", "Validation Result": "val_result", "Transaction Timestamp": "trans_time",
                  "Finished Timestamp": "finished_time", "TransactionID": "trans_id", 
                  "Transaction Details": "trans_det", " Amount " : "amt_sent", "Fee Charge" : "fee_1",
                  "Extra Fee Charge" : "fee_2", "Status" : "status", "Error Code" : "err_code", "Error Message" : "err_msg"}
# df.columns.rename([v for k,v in cols_corrected.items()], inplace = True)
df.rename(columns = cols_corrected, inplace = True)
df.columns

Index(['val_result', 'trans_time', 'finished_time', 'trans_id', 'trans_det',
       'amt_sent', 'fee_1', 'fee_2', 'status', 'err_code', 'err_msg'],
      dtype='object')

In [6]:
df["trans_time"]

Record No
 1.0     15042018 20:59:48
 2.0     15042018 20:59:48
 3.0     15042018 20:59:48
 4.0     15042018 20:59:48
 5.0     15042018 20:59:48
 6.0     15042018 20:59:49
 7.0     15042018 20:59:49
 8.0     15042018 20:59:49
 9.0     15042018 20:59:49
 10.0    15042018 20:59:49
 11.0    15042018 20:59:50
 12.0    15042018 20:59:50
 13.0    15042018 20:59:50
 14.0    15042018 20:59:50
 15.0    15042018 20:59:50
 16.0    15042018 20:59:51
 17.0    15042018 20:59:51
 18.0    15042018 20:59:51
 19.0    15042018 20:59:51
 20.0    15042018 20:59:51
 21.0    15042018 20:59:52
 22.0    15042018 20:59:52
 23.0    15042018 20:59:52
 24.0    15042018 20:59:52
 25.0    15042018 20:59:52
 26.0    15042018 20:59:53
 27.0    15042018 20:59:53
 28.0    15042018 20:59:53
 29.0    15042018 20:59:53
 30.0    15042018 20:59:53
               ...        
 59.0    15042018 20:59:59
 60.0    15042018 20:59:59
 61.0    15042018 21:00:00
 62.0    15042018 21:00:00
 63.0    15042018 21:00:00
 64.0    15042018 

In [7]:
df = df.loc[df.index.dropna()]

In [8]:
type(df.trans_time[1])

str

In [9]:
def convert(datestr):
#     year = int(datestr[4:8])
#     month = int(datestr[2:4])
#     day = int(datestr[0:2])
#     hour = int(datestr[9:11])
#     minute = int(datestr[12:14])
#     second = int(datestr[15:17])
    converted = dt.datetime.strptime(str(datestr),"%d%m%Y %H:%M:%S")
#    converted = pd.to_datetime(str(datestr), format="%d%m%Y %H:%M:%S", errors="raise")
    return converted

df.finished_time = df.finished_time.apply(convert)
df.trans_time = df.trans_time.apply(convert)

In [10]:
df.finished_time

Record No
1.0    2018-04-15 20:59:48
2.0    2018-04-15 20:59:48
3.0    2018-04-15 20:59:48
4.0    2018-04-15 20:59:48
5.0    2018-04-15 20:59:48
6.0    2018-04-15 20:59:49
7.0    2018-04-15 20:59:49
8.0    2018-04-15 20:59:49
9.0    2018-04-15 20:59:49
10.0   2018-04-15 20:59:49
11.0   2018-04-15 20:59:50
12.0   2018-04-15 20:59:50
13.0   2018-04-15 20:59:50
14.0   2018-04-15 20:59:50
15.0   2018-04-15 20:59:50
16.0   2018-04-15 20:59:51
17.0   2018-04-15 20:59:51
18.0   2018-04-15 20:59:51
19.0   2018-04-15 20:59:51
20.0   2018-04-15 20:59:51
21.0   2018-04-15 20:59:52
22.0   2018-04-15 20:59:52
23.0   2018-04-15 20:59:52
24.0   2018-04-15 20:59:52
25.0   2018-04-15 20:59:52
26.0   2018-04-15 20:59:53
27.0   2018-04-15 20:59:53
28.0   2018-04-15 20:59:53
29.0   2018-04-15 20:59:53
30.0   2018-04-15 20:59:53
               ...        
58.0   2018-04-15 20:59:59
59.0   2018-04-15 20:59:59
60.0   2018-04-15 20:59:59
61.0   2018-04-15 21:00:00
62.0   2018-04-15 21:00:00
63.0   2018-04-15 

In [11]:
df["year"] = df.finished_time.dt.year

In [12]:
df.year[1]

2018

In [13]:
df=df[["trans_id", "finished_time", "trans_det", "status", "amt_sent", "fee_1", "fee_2", "val_result", "trans_time", 
        "err_code", "err_msg"]]
df.columns

Index(['trans_id', 'finished_time', 'trans_det', 'status', 'amt_sent', 'fee_1',
       'fee_2', 'val_result', 'trans_time', 'err_code', 'err_msg'],
      dtype='object')

In [14]:
df.trans_id = df.trans_id.str[4:7]

In [15]:
df["bulk_plan_dets"] = bulk_plan_dets

In [16]:
df = df[df.status=="Completed"]

In [24]:
row_ct = len(df.finished_time)
year = str(df.finished_time.dt.year[row_ct])
month = "0" + str(df.finished_time.dt.month[row_ct]) if len(str(df.finished_time.dt.month[row_ct]))==1 else df.finished_time.dt.month[row_ct]
day = "0" + str(df.finished_time.dt.day[row_ct]) if len(str(df.finished_time.dt.day[row_ct]))==1 else df.finished_time.dt.day[row_ct]
# filename = str("-".join([str(df.finished_time.dt.year[row_ct]),
#                         str(df.finished_time.dt.month[row_ct]),
#                         str(df.finished_time.dt.day[row_ct])]))

In [25]:
year, month, day

(2018, '04', 15)

In [21]:
df.finished_time.dt.month[1]

4

In [32]:
# record = df.finished_time[1]
filename = str(df.finished_time.dt.year[1])+str(df.finished_time.dt.month[1])+str(df.finished_time.dt.day[1])
# pd.DataFrame.to_csv(df, filename)

In [36]:
filename = year + month + day + "-" + df.trans_id[row_ct] + ".csv"
# year, month, day

In [37]:
filename

'20180415-USI.csv'

In [ ]:
df.finished_time = pd.to_datetime(df.finished_time, infer_datetime_format=True)

In [ ]:
type(df.finished_time[1])

In [34]:
test = pd.DataFrame({'year': [2015, 2016],
                       'month': [2, 3],
                       'day': [4, 5]})

In [38]:
test = pd.to_datetime(test)

In [34]:
year = str(df.finished_time.dt.year[row_ct])
month = "0" + str(df.finished_time.dt.month[row_ct]) if (
            len(str(df.finished_time.dt.month[row_ct]))==1) else (
            str(df.finished_time.dt.month[row_ct]))
day = "0" + str(df.finished_time.dt.day[row_ct]) if (
            len(str(df.finished_time.dt.day[row_ct]))==1) else (
            str(df.finished_time.dt.day[row_ct]))

In [ ]:
!python mpesa_reformatter.py